# Individuel

In [1]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
from matplotlib.animation import FuncAnimation, FFMpegWriter

# Connect to DuckDB 
con = duckdb.connect("football.duckdb")

# Sécurité mémoire
con.execute("PRAGMA threads=4;")
con.execute("PRAGMA memory_limit='4GB';")

print("DuckDB ready")

con.execute("""
CREATE OR REPLACE TABLE summary AS
SELECT *
FROM read_csv_auto('data/summary.csv')
""")

DuckDB ready


## Tableau de zone

In [20]:
player_pseudo = 1      
month = '2020-03'            # format YYYY-MM
def speed_zone(player, date):
    df_speed = con.execute(f"""
    SELECT
        strftime('%Y-%m', session_date) AS month,
        player_pseudo,
        MAX(speed_kmh) AS max_speed_kmh
    FROM tracking_games
    WHERE player_pseudo = '{player_pseudo}'
    AND strftime('%Y-%m', session_date) = '{month}'
    GROUP BY month, player_pseudo
    """).df()
    zones = [0,0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    labels = [
        '0-40%',
        '40-50%',
        '50-60%',
        '60-70%',
        '70-80%',
        '80-90%',
        '90-100%'
    ]
    vmax = df_speed['max_speed_kmh'].values[0]

    df_zone = pd.DataFrame({
        'zone': labels,
        'min_speed_kmh': [vmax * z1 for z1 in zones[:-1]],
        'max_speed_kmh': [vmax * z2 for z2 in zones[1:]],
    })
    return df_zone
speed_zone(player_pseudo, month)


,zone,min_speed_kmh,max_speed_kmh
0,0-40%,0.000000,11.396304
1,40-50%,11.396304,14.245380
2,50-60%,14.245380,17.094456
3,60-70%,17.094456,19.943532
4,70-80%,19.943532,22.792608
5,80-90%,22.792608,25.641684
6,90-100%,25.641684,28.490760


## Profile accélération vitesse

Pour tracer le profil, il est proposé, à partir d’une vitesse de 3 m/s, de prendre toutes les 0,2 m/s les deux points dont l’accélération est maximale puis de calculer la droite de régression de l’accélération par rapport à la vitesse.
Miguens et al. [6] proposent une régression quantile dans le cadre du rugby.

In [ ]:
player = 41
match = '2020-02-01'

df_match_players = con.execute(f"""
SELECT *
FROM tracking_games
WHERE session_date = '{match}'
AND player_pseudo = {player}
""").df()